# Document Search

## 0. 미션
참조
- 위키백과:데이터베이스 다운로드: https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B2%A0%EC%9D%B4%EC%8A%A4_%EB%8B%A4%EC%9A%B4%EB%A1%9C%EB%93%9C
- wikiextractor: https://github.com/attardi/wikiextractor
- BM25: https://github.com/dorianbrown/rank_bm25
- SentenceTransformers: https://www.sbert.net/

미션
- 한국어 위키데이터를 이용해서 문서 데이터베이스를 구축한다.
- 입력된 키워드를 조건으로 BM25 알고리즘을 이용하여 관련된 문서를 검색한다.
- 입력된 키워드를 조건으로 인공지능 알고리즘을 이용하여 관련된 문서를 검색한다.

## 1. 라이브러리 설치 (최초 한번만 실행)
- 라이브러리는 colab이 최초 실행 또는 종료 후 실행된 경우 한번만 실행하면 됩니다.
- GPU 메모리 부족등의 이유로 colab 세션을 다시 시작한 경우는 설치할 필요 없습니다.
- colab 세션을 다시 시작하려면 '런타임' >> '세션 다시 시작'을 선택하세요.

In [1]:
!pip install -q -U transformers==4.38.2
!pip install -q -U datasets==2.18.0
!pip install -q -U bitsandbytes==0.42.0
!pip install -q -U peft==0.9.0
!pip install -q -U trl==0.7.11
!pip install -q -U accelerate==0.27.2
!pip install -q -U rank_bm25==0.2.2
!pip install -q -U sentence-transformers==2.7.0
!pip install -q -U wikiextractor==3.0.6
!pip install -q -U konlpy==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

## 2. 구글 드라이브 연결 (최초 한번만 실행)
- 구글 드라이브는 데이터 저장 및 학습 결과를 저장하기 위해서 사용합니다.
- 구글 드라이브는 colab이 최초 실행 또는 종료 후 실행된 경우 한번 만 연결하면 됩니다.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## *3. 환경 (매번 필수 실행)
- 환경은 colab 세션을 처음 시작하거나 다시 시작한 경우 실행되어야 합니다.
- 프로젝트 진행에 필요한 환경을 설정합니다.

### 3.1. 라이브러리 Import

In [3]:
import os
import glob
import json

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import konlpy
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer

### 3.2. 환경정보 설정
- WORKSPACE
  - 학습 데이터 및 학습결과를 저장하기 위한 경로입니다.
  - 필요할 경우 적당한 경로로 변경할 수 있습니다.
  - 경로를 변경 할 경우 전체 경로에 공백이 포함되지 않도록 주의해 주세요.
- MODEL_ID
  - 이번 프로젝트를 위한 SentenceTransformer 입니다.
  - 서울대학교 컴퓨터언어학_자연어처리 연구실에서 공개한 모델입니다.
  - https://huggingface.co/snunlp/KR-SBERT-V40K-klueNLI-augSTS
- CHUNK_FN
  - 문서를 일정한 단위로 분할해서 저장할 파일 이름
  - 데이터베이스를 대신하는 역할
  - 실제 기능을 만들 때는 DB를 사용해야 합니다.


In [4]:
WORKSPACE = '/content/drive/MyDrive/nlp-project'
MODEL_ID = 'snunlp/KR-SBERT-V40K-klueNLI-augSTS'
CHUNK_FN = os.path.join(WORKSPACE, "data", "chunk_db.json")

## 4. 한국어 위키 문서 데이터베이스 만들기
- 한국어 위키를 다운로드 하고, 전처리 해서 한국어 위키 문서 데이터베이스를 만드는 과정입니다.
- 이 과정은 한번만 완료 하면 됩니다.
- 전처리가 완료된 이후에는 더 이상 진행할 필요 없습니다.

### 4.1. 한국어 위키 덤프 최신파일 다운로드 하기
- 한국어 위키의 덤프 파일 중 가장 최신 파일을 다운로드 합니다.

In [ ]:
!wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-meta-current.xml.bz2 \
   -O {WORKSPACE}/data/kowiki-latest-pages-meta-current.xml.bz2

--2024-04-30 00:55:55--  https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-meta-current.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1195764746 (1.1G) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/nlp-project/data/kowiki-latest-pages-meta-current.xml.bz2’

/content/drive/MyDr 100%[===================>]   1.11G  4.65MB/s    in 4m 4s   

2024-04-30 01:00:00 (4.67 MB/s) - ‘/content/drive/MyDrive/nlp-project/data/kowiki-latest-pages-meta-current.xml.bz2’ saved [1195764746/1195764746]



### 4.2. 한국어 위키 전처리 하기
- WikiExtractor를 이용해서 한국어 위키를 전처리 하는 과정입니다.
- 출력 형태는 json으로 출력합니다.
- 시간이 오래 걸립니다. 결과는 '{WORKSPACE}/data/kowiki'에서 확인할 수 있습니다.

In [ ]:
!python -m wikiextractor.WikiExtractor \
        --json \
        --out {WORKSPACE}/data/kowiki \
        {WORKSPACE}/data/kowiki-latest-pages-meta-current.xml.bz2

INFO: Preprocessing '/content/drive/MyDrive/nlp-project/data/kowiki-latest-pages-meta-current.xml.bz2' to collect template definitions: this may take some time.
INFO: Preprocessed 100000 pages
INFO: Preprocessed 200000 pages
INFO: Preprocessed 300000 pages
INFO: Preprocessed 400000 pages
INFO: Preprocessed 500000 pages
INFO: Preprocessed 600000 pages
INFO: Preprocessed 700000 pages
INFO: Preprocessed 800000 pages
INFO: Preprocessed 900000 pages
INFO: Preprocessed 1000000 pages
INFO: Preprocessed 1100000 pages
INFO: Preprocessed 1200000 pages
INFO: Preprocessed 1300000 pages
INFO: Preprocessed 1400000 pages
INFO: Preprocessed 1500000 pages
INFO: Preprocessed 1600000 pages
INFO: Preprocessed 1700000 pages
INFO: Preprocessed 1800000 pages
INFO: Preprocessed 1900000 pages
INFO: Preprocessed 2000000 pages
INFO: Preprocessed 2100000 pages
INFO: Preprocessed 2200000 pages
INFO: Preprocessed 2300000 pages
INFO: Preprocessed 2400000 pages
INFO: Preprocessed 2500000 pages
INFO: Preprocessed 2600

In [ ]:
# 전처리 결과 확인
!ls {WORKSPACE}/data/kowiki

AA  AB	AC  AD	AE  AF	AG  AH	AI  AJ	AK  AL	AM  AN	AO  AP	AQ  AR	AS


In [ ]:
# 전처리 결과 확인
!ls {WORKSPACE}/data/kowiki/AA

wiki_00  wiki_10  wiki_20  wiki_30  wiki_40  wiki_50  wiki_60  wiki_70	wiki_80  wiki_90
wiki_01  wiki_11  wiki_21  wiki_31  wiki_41  wiki_51  wiki_61  wiki_71	wiki_81  wiki_91
wiki_02  wiki_12  wiki_22  wiki_32  wiki_42  wiki_52  wiki_62  wiki_72	wiki_82  wiki_92
wiki_03  wiki_13  wiki_23  wiki_33  wiki_43  wiki_53  wiki_63  wiki_73	wiki_83  wiki_93
wiki_04  wiki_14  wiki_24  wiki_34  wiki_44  wiki_54  wiki_64  wiki_74	wiki_84  wiki_94
wiki_05  wiki_15  wiki_25  wiki_35  wiki_45  wiki_55  wiki_65  wiki_75	wiki_85  wiki_95
wiki_06  wiki_16  wiki_26  wiki_36  wiki_46  wiki_56  wiki_66  wiki_76	wiki_86  wiki_96
wiki_07  wiki_17  wiki_27  wiki_37  wiki_47  wiki_57  wiki_67  wiki_77	wiki_87  wiki_97
wiki_08  wiki_18  wiki_28  wiki_38  wiki_48  wiki_58  wiki_68  wiki_78	wiki_88  wiki_98
wiki_09  wiki_19  wiki_29  wiki_39  wiki_49  wiki_59  wiki_69  wiki_79	wiki_89  wiki_99


In [ ]:
# 파일 내용을 5줄만 확인해 봅니다.
!head -n 5 {WORKSPACE}/data/kowiki/AA/wiki_00

{"id": "5", "revid": "36744442", "url": "https://ko.wikipedia.org/wiki?curid=5", "title": "\uc9c0\ubbf8 \uce74\ud130", "text": "\uc81c\uc784\uc2a4 \uc5bc \u201c\uc9c0\ubbf8\u201d \uce74\ud130 \uc8fc\ub2c8\uc5b4(, 1924\ub144 10\uc6d4 1\uc77c~)\ub294 \ubbfc\uc8fc\ub2f9 \ucd9c\uc2e0 \ubbf8\uad6d\uc758 \uc81c39\ub300 \ub300\ud1b5\ub839 (1977-81)\uc774\ub2e4.\n\uc0dd\uc560.\n\uc5b4\ub9b0 \uc2dc\uc808.\n\uc9c0\ubbf8 \uce74\ud130\ub294 \uc870\uc9c0\uc544\uc8fc \uc12c\ud130 \uce74\uc6b4\ud2f0 \ud50c\ub808\uc778\uc2a4 \ub9c8\uc744\uc5d0\uc11c \ud0dc\uc5b4\ub0ac\ub2e4.\n\uc870\uc9c0\uc544 \uacf5\uacfc\ub300\ud559\uad50\ub97c \uc878\uc5c5\ud558\uc600\ub2e4. \uadf8 \ud6c4 \ud574\uad70\uc5d0 \ub4e4\uc5b4\uac00 \uc804\ud568\u00b7\uc6d0\uc790\ub825\u00b7\uc7a0\uc218\ud568\uc758 \uc2b9\ubb34\uc6d0\uc73c\ub85c \uc77c\ud558\uc600\ub2e4. 1953\ub144 \ubbf8\uad6d \ud574\uad70 \ub300\uc704\ub85c \uc608\ud3b8\ud558\uc600\uace0 \uc774\ud6c4 \ub545\ucf69\u00b7\uba74\ud654 \ub4f1\uc744 \uac00\uafd4 \ub9ce\uc740

In [ ]:
# 5줄만 json 형태로 변경해서 출력해 봅니다.
with open(os.path.join(WORKSPACE, "data", "kowiki", "AA", "wiki_00")) as f:
    for i, line in enumerate(f):
        line = line.strip()
        # print(line)

        data = json.loads(line)
        print(data)

        if i >= 4:
            break

{'id': '5', 'revid': '36744442', 'url': 'https://ko.wikipedia.org/wiki?curid=5', 'title': '지미 카터', 'text': '제임스 얼 “지미” 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령 (1977-81)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n정계 입문.\n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n대통령 재임.\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책을 내세워서 많은 지지를 받았으며 제럴드 포드 대통령을 누르고 당선되었다.\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n외교 정책.\n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.\n임기 말, 

### 4.3. 문서 데이터베이스 만들기
- 문서를 일정 단위로 분할해서 문서 데이터베이스를 만들겠습니다.
- 별도의 DB는 사용하지 않고 json 형태로 저장하도록 하겠습니다.
- 문서의 길이가 다양하므로 긴 문서와 짧은 문서를 함께 처리하기 위해서, 띄어쓰기 기준으로 128단어가 나타나는 라인 단위로 잘라서 저장하겠습니다.

In [ ]:
fn_list = glob.glob(f"{WORKSPACE}/data/kowiki/*/*")
len(fn_list)

1805

In [ ]:
def make_chunk(text, n_word=128):
    # line 단위로 단어수 계산
    line_list = []
    total = 0
    for line in text.split('\n'):
        total += len(line.split())
        line_list.append((total, line))
    # n_word 단위로 분할
    chunk_list = []
    chunk_total, chunk_index = 0, 0
    for i, (total, line) in enumerate(line_list):
        if total - chunk_total >= n_word:
            chunk = [line for total, line in line_list[chunk_index:i+1]]
            chunk_list.append('\n'.join(chunk))
            chunk_index = i + 1
            chunk_total = total
    # 마지막 line 추가 (n_word 보다 작은 경우 이전라인 포함)
    if total > chunk_total:
        if total - chunk_total < n_word and chunk_index > 1:
            chunk_index -= 1
        chunk = [line for total, line in line_list[chunk_index:]]
        chunk_list.append('\n'.join(chunk))
    return chunk_list

In [ ]:
# 기능 확인을 위해서 문서를 chunk 단위로 분할해서 row_list에 저장
row_list = []
for fn in fn_list[:1]:
    with open(fn) as f:
        for line in f:
            data = json.loads(line)
            chunk_list = make_chunk(data['text'])
            for i, chunk in enumerate(chunk_list):
                title = data['title']
                row = {
                    'id': data['id'],
                    'chunk_id': str(i + 1),
                    'chunk': f"{title}\n{chunk}"
                }
                print(row)
                row_list.append(row)
len(row_list)

{'id': '5', 'chunk_id': '1', 'chunk': '지미 카터\n제임스 얼 “지미” 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령 (1977-81)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n정계 입문.\n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n대통령 재임.\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책을 내세워서 많은 지지를 받았으며 제럴드 포드 대통령을 누르고 당선되었다.\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.'}
{'id': '5', 'chunk_id': '2', 'chunk': '지미 카터\n외교 정책.\n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.\n임기 말, 소련의 아프가니스탄 침공 사

350

In [ ]:
# chunk를 CHUNK_FN에 저장합니다.
with open(CHUNK_FN, "w") as f:
    for row in row_list:
        f.write(json.dumps(row, ensure_ascii=False))
        f.write("\n")

In [ ]:
# CHUNK_FN의 내용을 확인합니다.
!head -n 5 {CHUNK_FN}

{"id": "5", "chunk_id": "1", "chunk": "지미 카터\n제임스 얼 “지미” 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령 (1977-81)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 \"땅콩 농부\" (Peanut Farmer)로 알려졌다.\n정계 입문.\n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n대통령 재임.\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책을 내세워서 많은 지지를 받았으며 제럴드 포드 대통령을 누르고 당선되었다.\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다."}
{"id": "5", "chunk_id": "2", "chunk": "지미 카터\n외교 정책.\n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.\n임기 말, 소련의 아프가니스탄 침공

## 5. BM25 검색 tutorial
- BM25를 이용해 관련 문서를 검색하는 방법에 대해서 알아봅니다.

### 5.1. bm25를 위한 전처리
- BM25를 위해서 tokenizer를 정의하고 전처리를 실행합니다.

In [5]:
# tutorial mini chunk list
chunk_list = [
    "지미 카터\n제임스 얼 “지미” 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령 (1977-81)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 \"땅콩 농부\" (Peanut Farmer)로 알려졌다.\n정계 입문.\n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n대통령 재임.\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책을 내세워서 많은 지지를 받았으며 제럴드 포드 대통령을 누르고 당선되었다.\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.",
    "수학\n수학(, , math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다. 널리 받아들여지는 명확한 정의는 없으나 현대 수학은 일반적으로 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 규칙의 발견과 문제의 제시 및 해결의 과정으로 이루어진다. 수학은 그 발전 과정에 있어서 철학, 과학과 깊은 연관을 맺고 있으며, 엄밀한 논리와 특유의 추상성, 보편성에 의해 다른 학문들과 구별된다. 특히 수학은 과학의 여느 분야들과는 달리 자연계에서 관측되지 않는 개념들에 대해서까지 이론을 추상화시키는 특징을 보이는데, 수학자들은 그러한 개념들에 대한 추측을 제시하고 적절하게 선택된 정의와 공리로부터 엄밀한 연역을 거쳐 그 진위를 파악한다.\n수학의 개념들은 기원전 600년 경에 활동하며 최초의 수학자로도 여겨지는 탈레스의 기록은 물론, 다른 고대 문명들에서도 찾아볼 수 있으며 인류의 문명과 함께 발전해 왔다. 오늘날 수학은 자연과학, 사회과학, 공학, 의학 등 거의 모든 학문에서도 핵심적인 역할을 하며 다양한 방식으로 응용된다.\n수학을 의미하는 mathematics라는 단어는 '아는 모든 것', '배우는 모든 것'이라는 뜻의 고대 그리스어 'máthēma'(μάθημα) 및 그 활용형 mathēmatikós(μαθηματικός)에서 유래되었다.",
    "수학 상수\n수학에서 상수란 그 값이 변하지 않는 불변량으로, 변수의 반대말이다. 물리 상수와는 달리, 수학 상수는 물리적 측정과는 상관없이 정의된다.\n수학 상수는 대개 실수체나 복소수체의 원소이다. 우리가 이야기할 수 있는 상수는 (거의 대부분 계산 가능한) 정의가능한 수이다.\n특정 수학 상수, 예를 들면 골롬-딕맨 상수, 프랑세즈-로빈슨 상수, formula_1, 레비 상수와 같은 상수는 다른 수학상수 또는 함수와 약한 상관관계 또는 강한 상관관계를 갖는다.",
    "문학\n문학(文學, )은 언어를 예술적 표현의 제재로 삼아 새로운 의미를 창출하여, 인간과 사회를 진실되게 묘사하는 예술의 하위분야이다. 간단하게 설명하면, 언어를 통해 인간의 삶을 미적(美的)으로 형상화한 것이라고 볼 수 있다. 문학은 원래 문예(文藝)라고 부르는 것이 옳으며, 문학을 학문의 대상으로서 탐구하는 학문의 명칭 역시 문예학이다. 문예학은 음악사학, 미술사학 등과 함께 예술학의 핵심분야로서 인문학의 하위범주에 포함된다.\n일반적으로 문학의 정의는 텍스트들의 집합이다. 각각의 국가들은 고유한 문학을 가질 수 있으며, 이는 기업이나 철학 조류, 어떤 특정한 역사적 시대도 마찬가지이다. 흔히 한 국가의 문학을 묶어서 분류한다. 예를 들어 고대 그리스어, 성서, 베오울프, 일리아드, 그리고 미국 헌법 등이 그러한 분류의 범주에 들어간다. 좀 더 일반적으로는 문학은 특정한 주제를 가진 이야기, 시, 희곡의 모음이라 할 수 있다. 이 경우, 이야기, 시, 그리고 희곡은 민족주의적인 색채를 띨 수도 아닐 수도 있다. 문학의 한 부분으로서 특정한 아이템을 구분 짓는 일은 매우 어려운 일이다. 어떤 사람들에게 \"문학\"은 어떠한 상징적인 기록의 형태로도 나타날 수 있는 것이다. (이를테면 이미지나 조각, 또는 문자로도 나타날 수 있다.) 그러나 또다른 사람들에게 있어 문학은 오직 문자로 이루어진 텍스트로 구성된 것만을 포함한다. 좀 더 보수적인 사람들은 그 개념이 꼭 물리적인 형태를 가진 텍스트여야 하고, 대개 그러한 형태는 종이 등의 눈에 보이는 매체에서 디지털 미디어까지 다양할 수 있다.",
    "화학\n화학(化學)은 물질의 성질, 조성, 구조, 변화 및 그에 수반하는 에너지의 변화를 연구하는 자연과학(自然科學)의 한 분야이다. 물리학(物理學)도 역시 물질을 다루는 학문이지만, 물리학이 원소(元素)와 화합물(化合物)을 모두 포함한 물체의 운동과 에너지, 열적·전기적·광학적·기계적 속성을 다루고 이러한 현상으로부터 통일된 이론을 구축하려는 것과는 달리 화학에서는 물질 자체를 연구 대상으로 한다. 화학은 이미 존재하는 물질을 이용하여 특정한 목적에 맞는 새로운 물질을 합성하는 길을 제공하며, 이는 농작물(農作物)의 증산, 질병의 치료 및 예방, 에너지 효율 증대, 환경오염(環境汚染) 감소 등 여러 가지 이점을 제공한다.\n어원.\n화학은 연금술사들이 물질을 섞으며 발전시켰기 때문에 화학을 뜻하는 영어 ‘케미스트리(chemistry)’는 연금술을 뜻하는 단어 ‘알케미(alchemy)’에서 비롯하였다. 이는 다시 아랍어 ‘알 키미야(, al-kīmiyāʾ)’에서 왔는데, 이 단어의 어원에 대해서는 여러 가지 설이 있다.\n‘화학(化學)’이란 단어는 물질의 변화를 다루는 학문이라는 점에 착안한 번역어이다. 이 번역어는 의 《항해술기(航海述奇)》(1866), 의 자연과학 교과서 《격물입문(格物入門)》(1866) 등에서 처음 쓰였다.\n역사.\n고대 화학(古代化學)",
]

In [6]:
# 형태소 분석기를 이용한 tokeinizer 선언
# Komoran 품사표: https://docs.komoran.kr/firststep/postypes.html
KOMORAN = konlpy.tag.Komoran()
EXCLUDE = set(['JC', 'JKB', 'JKC', 'JKG', 'JKO', 'JKQ', 'JKS', 'JKV', 'JX',
               'EC', 'EF', 'EP', 'ETM', 'ETN'])
def tokenizer(sent):
    tokens = []
    for w, t in KOMORAN.pos(sent):
        if t not in EXCLUDE:
            tokens.append(w)
    return tokens

In [7]:
# tokenize
tokenized_chunks = [tokenizer(chunk) for chunk in chunk_list]
print(tokenized_chunks[0])

['지미 카터', '제임스', '얼', '“', '지미', '”', '카터', '주니어', '(', ',', '1924', '년', '10월 1일', '~', ')', '민주당', '출신', '미국', '제', '39', '대', '대통령', '(', '1977', '-', '81', ')', '이', '.', '생애', '.', '어리', '시절', '.', '지미 카터', '조지', '아주', '섬터', '카운티', '플레인스', '마을', '태어나', '.', '조지아 공과대학교', '졸업', '하', '.', '그 후', '해군', '들어가', '전함', '·', '원자력', '·', '잠수함', '승무원', '일', '하', '.', '1953', '년', '미국 해군', '대위', '예편', '하', '이후', '땅콩', '·', '면', '화', '등', '가꾸', '많', '돈', '벌', '.', '그', '별명', '"', '땅콩', '농부', '"', '(', 'Peanut', 'Farmer', ')', '알리', '지', '.', '정계', '입문', '.', '1962', '년', '조지', '아주', '상원', '의원', '선거', '낙선', '하', ',', '그', '선거', '부정', '선거', '이', '입증', '하', '되', '당선', '되', ',', '1966', '년', '조지아 주', '지사', '선거', '낙선', '하', ',', '1970', '년', '조지아 주', '지사', '선거', '당선', '되', '.', '대통령', '되', '전', '조지', '아주', '상원', '의원', '두', '번', '연임', '하', ',', '1971', '년', '1975', '년', '조지아', '지사', '근무', '하', '.', '조지아 주', '지사', '지내', ',', '미국', '살', '흑인', '등용', '법', '내세우', '.', '대통령', '재임', '.', '1976', '년', '미', 

### 5.2. tutorial step by step
- bm25를 활용한 검색 기능을 알아봅니다.

In [8]:
# bm25 api 생성
bm25 = BM25Okapi(tokenized_chunks)

In [9]:
# 각 문서의 token 길이
bm25.doc_len

[198, 241, 89, 282, 252]

In [10]:
# 문서별 term freq
bm25.doc_freqs

[{'지미 카터': 2,
  '제임스': 1,
  '얼': 1,
  '“': 1,
  '지미': 1,
  '”': 1,
  '카터': 2,
  '주니어': 1,
  '(': 3,
  ',': 6,
  '1924': 1,
  '년': 8,
  '10월 1일': 1,
  '~': 1,
  ')': 3,
  '민주당': 2,
  '출신': 1,
  '미국': 2,
  '제': 2,
  '39': 2,
  '대': 2,
  '대통령': 5,
  '1977': 1,
  '-': 1,
  '81': 1,
  '이': 2,
  '.': 15,
  '생애': 1,
  '어리': 1,
  '시절': 1,
  '조지': 3,
  '아주': 3,
  '섬터': 1,
  '카운티': 1,
  '플레인스': 1,
  '마을': 1,
  '태어나': 1,
  '조지아 공과대학교': 1,
  '졸업': 1,
  '하': 10,
  '그 후': 1,
  '해군': 1,
  '들어가': 1,
  '전함': 1,
  '·': 3,
  '원자력': 1,
  '잠수함': 1,
  '승무원': 1,
  '일': 1,
  '1953': 1,
  '미국 해군': 1,
  '대위': 1,
  '예편': 1,
  '이후': 1,
  '땅콩': 2,
  '면': 1,
  '화': 1,
  '등': 1,
  '가꾸': 1,
  '많': 2,
  '돈': 1,
  '벌': 1,
  '그': 2,
  '별명': 1,
  '"': 2,
  '농부': 1,
  'Peanut': 1,
  'Farmer': 1,
  '알리': 1,
  '지': 1,
  '정계': 1,
  '입문': 1,
  '1962': 1,
  '상원': 2,
  '의원': 2,
  '선거': 5,
  '낙선': 2,
  '부정': 1,
  '입증': 1,
  '되': 6,
  '당선': 3,
  '1966': 1,
  '조지아 주': 3,
  '지사': 4,
  '1970': 1,
  '전': 1,
  '두': 1,
  '번': 1,
  '연임'

In [11]:
# inverse document freq
bm25.idf

{'지미 카터': 1.0986122886681098,
 '제임스': 1.0986122886681098,
 '얼': 1.0986122886681098,
 '“': 1.0986122886681098,
 '지미': 1.0986122886681098,
 '”': 1.0986122886681098,
 '카터': 1.0986122886681098,
 '주니어': 1.0986122886681098,
 '(': 0.2261719586388148,
 ',': 0.2261719586388148,
 '1924': 1.0986122886681098,
 '년': 1.0986122886681098,
 '10월 1일': 1.0986122886681098,
 '~': 1.0986122886681098,
 ')': 0.2261719586388148,
 '민주당': 1.0986122886681098,
 '출신': 1.0986122886681098,
 '미국': 0.33647223662121295,
 '제': 1.0986122886681098,
 '39': 1.0986122886681098,
 '대': 1.0986122886681098,
 '대통령': 1.0986122886681098,
 '1977': 1.0986122886681098,
 '-': 0.2261719586388148,
 '81': 1.0986122886681098,
 '이': 0.2261719586388148,
 '.': 0.2261719586388148,
 '생애': 1.0986122886681098,
 '어리': 1.0986122886681098,
 '시절': 1.0986122886681098,
 '조지': 1.0986122886681098,
 '아주': 1.0986122886681098,
 '섬터': 1.0986122886681098,
 '카운티': 1.0986122886681098,
 '플레인스': 1.0986122886681098,
 '마을': 1.0986122886681098,
 '태어나': 1.098612288668

In [12]:
# 검색어를 이용한 검색
query = "수학교과서"
tokenized_query = tokenizer(query)
print(tokenized_query)

doc_scores = bm25.get_scores(tokenized_query)
doc_scores

['수학', '교과서']


array([0.        , 0.69724463, 0.59104911, 0.        , 1.01357506])

In [13]:
# score 순서로 정렬
rank = np.argsort(-doc_scores)
rank

array([4, 1, 2, 0, 3])

In [14]:
# top-3 출력
for r, i in enumerate(rank[:3]):
    print(f'---- rank: {r+1}, score: {doc_scores[i]} ----')
    print(chunk_list[i])
    print()

---- rank: 1, score: 1.0135750591308597 ----
화학
화학(化學)은 물질의 성질, 조성, 구조, 변화 및 그에 수반하는 에너지의 변화를 연구하는 자연과학(自然科學)의 한 분야이다. 물리학(物理學)도 역시 물질을 다루는 학문이지만, 물리학이 원소(元素)와 화합물(化合物)을 모두 포함한 물체의 운동과 에너지, 열적·전기적·광학적·기계적 속성을 다루고 이러한 현상으로부터 통일된 이론을 구축하려는 것과는 달리 화학에서는 물질 자체를 연구 대상으로 한다. 화학은 이미 존재하는 물질을 이용하여 특정한 목적에 맞는 새로운 물질을 합성하는 길을 제공하며, 이는 농작물(農作物)의 증산, 질병의 치료 및 예방, 에너지 효율 증대, 환경오염(環境汚染) 감소 등 여러 가지 이점을 제공한다.
어원.
화학은 연금술사들이 물질을 섞으며 발전시켰기 때문에 화학을 뜻하는 영어 ‘케미스트리(chemistry)’는 연금술을 뜻하는 단어 ‘알케미(alchemy)’에서 비롯하였다. 이는 다시 아랍어 ‘알 키미야(, al-kīmiyāʾ)’에서 왔는데, 이 단어의 어원에 대해서는 여러 가지 설이 있다.
‘화학(化學)’이란 단어는 물질의 변화를 다루는 학문이라는 점에 착안한 번역어이다. 이 번역어는 의 《항해술기(航海述奇)》(1866), 의 자연과학 교과서 《격물입문(格物入門)》(1866) 등에서 처음 쓰였다.
역사.
고대 화학(古代化學)

---- rank: 2, score: 0.6972446303817912 ----
수학
수학(, , math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다. 널리 받아들여지는 명확한 정의는 없으나 현대 수학은 일반적으로 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 규칙의 발견과 문제의 제시 및 해결의 과정으로 이루어진다. 수학은 그 발전 과정에 있어서 철학, 과학과 깊은 연관을 맺고 있으며, 엄밀한 논리와 특유의 추상성, 보편성에 의해 다른 학문들과 구별된다. 특히 수학은 과학의 여느 분야들과는 달리 자연계에서

### 5.3. tutorial 함수
- bm25를 활용한 검색 기능을 함수로 구현하고 실험합니다.

In [15]:
# bm25 api 생성
bm25 = BM25Okapi(tokenized_chunks)

In [16]:
def query_bm25(bm25, query, tokenizer, top_n=10):
    tokenized_query = tokenizer(query)
    # score 계산
    doc_scores = bm25.get_scores(tokenized_query)
    # score 순서로 정렬
    rank = np.argsort(-doc_scores)
    # top-n
    result = []
    for i in rank[:top_n]:
        if doc_scores[i] > 0:
            result.append((i, doc_scores[i]))
    return result

In [ ]:
while True:
    query = input('검색 > ')
    query = query.strip()
    if len(query) == 0:
        break
    result = query_bm25(bm25, query, tokenizer)
    for i, score in result:
        print(f'---- score: {score} ----')
        print(chunk_list[i])
        print()

검색 > 수학 교과서
---- score: 1.0135750591308597 ----
화학
화학(化學)은 물질의 성질, 조성, 구조, 변화 및 그에 수반하는 에너지의 변화를 연구하는 자연과학(自然科學)의 한 분야이다. 물리학(物理學)도 역시 물질을 다루는 학문이지만, 물리학이 원소(元素)와 화합물(化合物)을 모두 포함한 물체의 운동과 에너지, 열적·전기적·광학적·기계적 속성을 다루고 이러한 현상으로부터 통일된 이론을 구축하려는 것과는 달리 화학에서는 물질 자체를 연구 대상으로 한다. 화학은 이미 존재하는 물질을 이용하여 특정한 목적에 맞는 새로운 물질을 합성하는 길을 제공하며, 이는 농작물(農作物)의 증산, 질병의 치료 및 예방, 에너지 효율 증대, 환경오염(環境汚染) 감소 등 여러 가지 이점을 제공한다.
어원.
화학은 연금술사들이 물질을 섞으며 발전시켰기 때문에 화학을 뜻하는 영어 ‘케미스트리(chemistry)’는 연금술을 뜻하는 단어 ‘알케미(alchemy)’에서 비롯하였다. 이는 다시 아랍어 ‘알 키미야(, al-kīmiyāʾ)’에서 왔는데, 이 단어의 어원에 대해서는 여러 가지 설이 있다.
‘화학(化學)’이란 단어는 물질의 변화를 다루는 학문이라는 점에 착안한 번역어이다. 이 번역어는 의 《항해술기(航海述奇)》(1866), 의 자연과학 교과서 《격물입문(格物入門)》(1866) 등에서 처음 쓰였다.
역사.
고대 화학(古代化學)

---- score: 0.6972446303817912 ----
수학
수학(, , math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다. 널리 받아들여지는 명확한 정의는 없으나 현대 수학은 일반적으로 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 규칙의 발견과 문제의 제시 및 해결의 과정으로 이루어진다. 수학은 그 발전 과정에 있어서 철학, 과학과 깊은 연관을 맺고 있으며, 엄밀한 논리와 특유의 추상성, 보편성에 의해 다른 학문들과 구별된다. 특히 수학은 과학의 여느 분야들과는 달리 자연계에서 관측되지 

## 6. BM25 검색 실습
- BM25를 이용해 관련 문서를 검색 기능을 직접 구현하는 과정입니다.

### 6.1. bm25를 위한 전처리
- CHUNK_FN의 전체 문서를 사용하세요.
- BM25를 위해서 tokenizer를 정의하고 전처리를 실행합니다.

In [19]:
!ls

drive  sample_data


In [20]:
CHUNK_FN

'/content/drive/MyDrive/nlp-project/data/chunk_db.json'

In [28]:
# 빈 리스트 생성
data_list = []

# JSON 파일을 줄 단위로 읽어오기
with open(CHUNK_FN, 'r', encoding='utf-8') as f:
    for line in f:
        # 각 줄을 JSON 객체로 변환하여 리스트에 추가
        data = json.loads(line)
        data_list.append(data)

# 데이터 프레임으로 변환
df = pd.DataFrame(data_list)

df.head()

,id,chunk_id,chunk
0,5,1,"지미 카터\n제임스 얼 “지미” 카터 주니어(, 1924년 10월 1일~)는 민주당..."
1,5,2,지미 카터\n외교 정책.\n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안...
2,5,3,지미 카터\n1979년~1980년 대한민국의 정치적 격변기 당시의 대통령이었던 그는...
3,5,4,"지미 카터\n특히 국제 분쟁 조정을 위해 북한의 김일성, 아이티의 세드라스 장군, ..."
4,5,5,지미 카터\n2011년 4월 26일부터 29일까지 북한을 3일간 방문했다.\n201...


In [30]:
chunk_list = df['chunk'].to_list()
chunk_list

['지미 카터\n제임스 얼 “지미” 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령 (1977-81)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n정계 입문.\n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n대통령 재임.\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책을 내세워서 많은 지지를 받았으며 제럴드 포드 대통령을 누르고 당선되었다.\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.',
 '지미 카터\n외교 정책.\n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.\n임기 말, 소련의 아프가니스탄 침공 사건으로 인해 1980년 하계 올림픽에 반공국가들의 보이콧을 하였다.\n그는 주이란 미국 대사관 인질 사건의 인질 구출 실패로 인한 원

In [31]:
# tokenize
tokenized_chunks = [tokenizer(chunk) for chunk in chunk_list]
print(tokenized_chunks[0])

['지미 카터', '제임스', '얼', '“', '지미', '”', '카터', '주니어', '(', ',', '1924', '년', '10월 1일', '~', ')', '민주당', '출신', '미국', '제', '39', '대', '대통령', '(', '1977', '-', '81', ')', '이', '.', '생애', '.', '어리', '시절', '.', '지미 카터', '조지', '아주', '섬터', '카운티', '플레인스', '마을', '태어나', '.', '조지아 공과대학교', '졸업', '하', '.', '그 후', '해군', '들어가', '전함', '·', '원자력', '·', '잠수함', '승무원', '일', '하', '.', '1953', '년', '미국 해군', '대위', '예편', '하', '이후', '땅콩', '·', '면', '화', '등', '가꾸', '많', '돈', '벌', '.', '그', '별명', '"', '땅콩', '농부', '"', '(', 'Peanut', 'Farmer', ')', '알리', '지', '.', '정계', '입문', '.', '1962', '년', '조지', '아주', '상원', '의원', '선거', '낙선', '하', ',', '그', '선거', '부정', '선거', '이', '입증', '하', '되', '당선', '되', ',', '1966', '년', '조지아 주', '지사', '선거', '낙선', '하', ',', '1970', '년', '조지아 주', '지사', '선거', '당선', '되', '.', '대통령', '되', '전', '조지', '아주', '상원', '의원', '두', '번', '연임', '하', ',', '1971', '년', '1975', '년', '조지아', '지사', '근무', '하', '.', '조지아 주', '지사', '지내', ',', '미국', '살', '흑인', '등용', '법', '내세우', '.', '대통령', '재임', '.', '1976', '년', '미', 

In [32]:
len(tokenized_chunks)

350

In [33]:
# bm25 api 생성
bm25 = BM25Okapi(tokenized_chunks)

### 6.2. BM25 검색 함수
- bm25를 활용한 검색 기능을 함수로 구현하고 실험합니다.

In [34]:
def query_bm25(bm25, query, tokenizer, top_n=10):
    tokenized_query = tokenizer(query)
    # score 계산
    doc_scores = bm25.get_scores(tokenized_query)
    # score 순서로 정렬
    rank = np.argsort(-doc_scores)
    # top-n
    result = []
    for i in rank[:top_n]:
        if doc_scores[i] > 0:
            result.append((i, doc_scores[i]))
    return result

In [35]:
while True:
    query = input('검색 > ')
    query = query.strip()
    if len(query) == 0:
        break
    result = query_bm25(bm25, query, tokenizer)
    for i, score in result:
        print(f'---- score: {score} ----')
        print(chunk_list[i])
        print()

검색 > 수학의 정의
---- score: 5.530034825502101 ----
수학
역사.
역사적으로 고대부터 현대에 이르기까지 문명에 필수적인 건축, 천문학, 정치, 상업 등에 수학적 개념들이 응용되어 왔다. 교역·분배·과세 등 인류의 사회 생활에 필요한 모든 계산에 수학이 관여해 왔고, 농경 생활에 필수적인 천문 관측과 달력의 제정, 토지의 측량 또한 수학이 직접적으로 사용된 분야이다. 고대 수학을 크게 발전시킨 문명으로는 메소포타미아, 이집트, 인도, 중국, 그리스 등이 있다.
특히 고대 그리스 문명에서는 처음으로 방정식에서 변수를 문자로 쓰는 등 추상화가 발전하였고 유클리드의 원론에서는 최초로 엄밀한 논증에 근거한 수학이 나타난다. 수학의 발전은 이후로도 계속되어 16세기의 르네상스에 이르러서는 과학적 방법과의 상호 작용을 통해 수학과 자연과학에 있어서 혁명적인 연구들이 진척되었고, 이는 인류 문명 발달에 큰 영향을 미치게 되었다.
세부 분야.
수학의 각 분야들은 상업에 필요한 계산을 하기 위해, 숫자들의 관계를 이해하기 위해, 토지를 측량하기 위해, 그리고 천문학적 사건들을 예견하기 위해 발전되어왔다. 이 네 가지 목적은 대략적으로 수학이 다루는 대상인 양, 구조, 공간 및 변화에 대응되며, 이들을 다루는 수학의 분야를 각각 산술, 대수학, 기하학, 해석학이라 한다. 또한 이 밖에도 근대 이후에 나타난 수학기초론과 이산수학 및 응용수학 등이 있다.

---- score: 5.4672796502298695 ----
수학
수학기초론 관련 분야.
수학의 기초를 확실히 세우기 위해, 수리논리학과 집합론이 발전하였고, 이와 더불어 범주론이 최근에도 발전되고 있다. “근본 위기”라는 말은 대략 1900년에서 1930년 사이에 일어난, 수학의 엄밀한 기초에 대한 탐구를 상징적으로 보여주는 말이다. 수학의 엄밀한 기초에 대한 몇 가지 의견 불일치는 오늘날에도 계속되고 있다. 수학의 기초에 대한 위기는 그 당시 수많은 논쟁에 의해 촉발되었으며, 그 논쟁에는 칸토

## 7. SentenceTransformers 검색 tutorial
- SentenceTransformers를 이용해 관련 문서를 검색하는 방법에 대해서 알아봅니다.

### 7.1. SentenceTransformers를 위한 데이터셋 정의
- SentenceTransformers 사용을 위해 데이터셋을 정의합니다.
- 별도의 전처리는 필요하지 않습니다.

In [ ]:
# tutorial mini chunk list
chunk_list = [
    "지미 카터\n제임스 얼 “지미” 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령 (1977-81)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 \"땅콩 농부\" (Peanut Farmer)로 알려졌다.\n정계 입문.\n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n대통령 재임.\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책을 내세워서 많은 지지를 받았으며 제럴드 포드 대통령을 누르고 당선되었다.\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.",
    "수학\n수학(, , math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다. 널리 받아들여지는 명확한 정의는 없으나 현대 수학은 일반적으로 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 규칙의 발견과 문제의 제시 및 해결의 과정으로 이루어진다. 수학은 그 발전 과정에 있어서 철학, 과학과 깊은 연관을 맺고 있으며, 엄밀한 논리와 특유의 추상성, 보편성에 의해 다른 학문들과 구별된다. 특히 수학은 과학의 여느 분야들과는 달리 자연계에서 관측되지 않는 개념들에 대해서까지 이론을 추상화시키는 특징을 보이는데, 수학자들은 그러한 개념들에 대한 추측을 제시하고 적절하게 선택된 정의와 공리로부터 엄밀한 연역을 거쳐 그 진위를 파악한다.\n수학의 개념들은 기원전 600년 경에 활동하며 최초의 수학자로도 여겨지는 탈레스의 기록은 물론, 다른 고대 문명들에서도 찾아볼 수 있으며 인류의 문명과 함께 발전해 왔다. 오늘날 수학은 자연과학, 사회과학, 공학, 의학 등 거의 모든 학문에서도 핵심적인 역할을 하며 다양한 방식으로 응용된다.\n수학을 의미하는 mathematics라는 단어는 '아는 모든 것', '배우는 모든 것'이라는 뜻의 고대 그리스어 'máthēma'(μάθημα) 및 그 활용형 mathēmatikós(μαθηματικός)에서 유래되었다.",
    "수학 상수\n수학에서 상수란 그 값이 변하지 않는 불변량으로, 변수의 반대말이다. 물리 상수와는 달리, 수학 상수는 물리적 측정과는 상관없이 정의된다.\n수학 상수는 대개 실수체나 복소수체의 원소이다. 우리가 이야기할 수 있는 상수는 (거의 대부분 계산 가능한) 정의가능한 수이다.\n특정 수학 상수, 예를 들면 골롬-딕맨 상수, 프랑세즈-로빈슨 상수, formula_1, 레비 상수와 같은 상수는 다른 수학상수 또는 함수와 약한 상관관계 또는 강한 상관관계를 갖는다.",
    "문학\n문학(文學, )은 언어를 예술적 표현의 제재로 삼아 새로운 의미를 창출하여, 인간과 사회를 진실되게 묘사하는 예술의 하위분야이다. 간단하게 설명하면, 언어를 통해 인간의 삶을 미적(美的)으로 형상화한 것이라고 볼 수 있다. 문학은 원래 문예(文藝)라고 부르는 것이 옳으며, 문학을 학문의 대상으로서 탐구하는 학문의 명칭 역시 문예학이다. 문예학은 음악사학, 미술사학 등과 함께 예술학의 핵심분야로서 인문학의 하위범주에 포함된다.\n일반적으로 문학의 정의는 텍스트들의 집합이다. 각각의 국가들은 고유한 문학을 가질 수 있으며, 이는 기업이나 철학 조류, 어떤 특정한 역사적 시대도 마찬가지이다. 흔히 한 국가의 문학을 묶어서 분류한다. 예를 들어 고대 그리스어, 성서, 베오울프, 일리아드, 그리고 미국 헌법 등이 그러한 분류의 범주에 들어간다. 좀 더 일반적으로는 문학은 특정한 주제를 가진 이야기, 시, 희곡의 모음이라 할 수 있다. 이 경우, 이야기, 시, 그리고 희곡은 민족주의적인 색채를 띨 수도 아닐 수도 있다. 문학의 한 부분으로서 특정한 아이템을 구분 짓는 일은 매우 어려운 일이다. 어떤 사람들에게 \"문학\"은 어떠한 상징적인 기록의 형태로도 나타날 수 있는 것이다. (이를테면 이미지나 조각, 또는 문자로도 나타날 수 있다.) 그러나 또다른 사람들에게 있어 문학은 오직 문자로 이루어진 텍스트로 구성된 것만을 포함한다. 좀 더 보수적인 사람들은 그 개념이 꼭 물리적인 형태를 가진 텍스트여야 하고, 대개 그러한 형태는 종이 등의 눈에 보이는 매체에서 디지털 미디어까지 다양할 수 있다.",
    "화학\n화학(化學)은 물질의 성질, 조성, 구조, 변화 및 그에 수반하는 에너지의 변화를 연구하는 자연과학(自然科學)의 한 분야이다. 물리학(物理學)도 역시 물질을 다루는 학문이지만, 물리학이 원소(元素)와 화합물(化合物)을 모두 포함한 물체의 운동과 에너지, 열적·전기적·광학적·기계적 속성을 다루고 이러한 현상으로부터 통일된 이론을 구축하려는 것과는 달리 화학에서는 물질 자체를 연구 대상으로 한다. 화학은 이미 존재하는 물질을 이용하여 특정한 목적에 맞는 새로운 물질을 합성하는 길을 제공하며, 이는 농작물(農作物)의 증산, 질병의 치료 및 예방, 에너지 효율 증대, 환경오염(環境汚染) 감소 등 여러 가지 이점을 제공한다.\n어원.\n화학은 연금술사들이 물질을 섞으며 발전시켰기 때문에 화학을 뜻하는 영어 ‘케미스트리(chemistry)’는 연금술을 뜻하는 단어 ‘알케미(alchemy)’에서 비롯하였다. 이는 다시 아랍어 ‘알 키미야(, al-kīmiyāʾ)’에서 왔는데, 이 단어의 어원에 대해서는 여러 가지 설이 있다.\n‘화학(化學)’이란 단어는 물질의 변화를 다루는 학문이라는 점에 착안한 번역어이다. 이 번역어는 의 《항해술기(航海述奇)》(1866), 의 자연과학 교과서 《격물입문(格物入門)》(1866) 등에서 처음 쓰였다.\n역사.\n고대 화학(古代化學)",
]

### 7.2. tutorial step by step
- SentenceTransformers를 활용한 검색 기능을 알아봅니다.

In [ ]:
# SentenceTransformer 모델 생성
model = SentenceTransformer(MODEL_ID)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# chunk embeddings 생성
chunk_embeddings = model.encode(chunk_list)
chunk_embeddings.shape

(5, 768)

In [ ]:
# 검색어를 이용한 검색
query = "수학교과서"
query_embedding = model.encode([query])
print(query_embedding.shape)

doc_scores = np.matmul(chunk_embeddings, query_embedding.T)
doc_scores = doc_scores.reshape(-1)
doc_scores

(1, 768)


array([  9.5166025, 119.72266  , 106.59877  ,  45.742844 ,  41.361908 ],
      dtype=float32)

In [ ]:
# score 순서로 정렬
rank = np.argsort(-doc_scores)
rank

array([1, 2, 3, 4, 0])

In [ ]:
# top-3 출력
for r, i in enumerate(rank[:3]):
    print(f'---- rank: {r+1}, score: {doc_scores[i]} ----')
    print(chunk_list[i])
    print()

---- rank: 1, score: 119.72265625 ----
수학
수학(, , math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다. 널리 받아들여지는 명확한 정의는 없으나 현대 수학은 일반적으로 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 규칙의 발견과 문제의 제시 및 해결의 과정으로 이루어진다. 수학은 그 발전 과정에 있어서 철학, 과학과 깊은 연관을 맺고 있으며, 엄밀한 논리와 특유의 추상성, 보편성에 의해 다른 학문들과 구별된다. 특히 수학은 과학의 여느 분야들과는 달리 자연계에서 관측되지 않는 개념들에 대해서까지 이론을 추상화시키는 특징을 보이는데, 수학자들은 그러한 개념들에 대한 추측을 제시하고 적절하게 선택된 정의와 공리로부터 엄밀한 연역을 거쳐 그 진위를 파악한다.
수학의 개념들은 기원전 600년 경에 활동하며 최초의 수학자로도 여겨지는 탈레스의 기록은 물론, 다른 고대 문명들에서도 찾아볼 수 있으며 인류의 문명과 함께 발전해 왔다. 오늘날 수학은 자연과학, 사회과학, 공학, 의학 등 거의 모든 학문에서도 핵심적인 역할을 하며 다양한 방식으로 응용된다.
수학을 의미하는 mathematics라는 단어는 '아는 모든 것', '배우는 모든 것'이라는 뜻의 고대 그리스어 'máthēma'(μάθημα) 및 그 활용형 mathēmatikós(μαθηματικός)에서 유래되었다.

---- rank: 2, score: 106.59877014160156 ----
수학 상수
수학에서 상수란 그 값이 변하지 않는 불변량으로, 변수의 반대말이다. 물리 상수와는 달리, 수학 상수는 물리적 측정과는 상관없이 정의된다.
수학 상수는 대개 실수체나 복소수체의 원소이다. 우리가 이야기할 수 있는 상수는 (거의 대부분 계산 가능한) 정의가능한 수이다.
특정 수학 상수, 예를 들면 골롬-딕맨 상수, 프랑세즈-로빈슨 상수, formula_1, 레비 상수와 같은 상수는 다른 수학상수 또는 함수와 약한 상관관계 또는 강한 상관관계를 갖는다.



### 7.3. tutorial 함수
- SentenceTransformer를 활용한 검색 기능을 함수로 구현하고 실험합니다.

In [ ]:
# SentenceTransformer 모델 생성
model = SentenceTransformer(MODEL_ID)

In [ ]:
# chunk embeddings 생성
chunk_embeddings = model.encode(chunk_list)
chunk_embeddings.shape

(5, 768)

In [ ]:
def query_sentence_transformer(model, chunk_embeddings, query, top_n=10):
    query_embedding = model.encode([query])
    # score 계산
    doc_scores = np.matmul(chunk_embeddings, query_embedding.T)
    doc_scores = doc_scores.reshape(-1)
    # score 순서로 정렬
    rank = np.argsort(-doc_scores)
    # top-n
    result = []
    for i in rank[:top_n]:
        result.append((i, doc_scores[i]))
    return result

In [ ]:
while True:
    query = input('검색 > ')
    query = query.strip()
    if len(query) == 0:
        break
    result = query_sentence_transformer(model, chunk_embeddings, query)
    for i, score in result:
        print(f'---- score: {score} ----')
        print(chunk_list[i])
        print()

검색 > 수학 교과서
---- score: 126.64801788330078 ----
수학
수학(, , math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다. 널리 받아들여지는 명확한 정의는 없으나 현대 수학은 일반적으로 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 규칙의 발견과 문제의 제시 및 해결의 과정으로 이루어진다. 수학은 그 발전 과정에 있어서 철학, 과학과 깊은 연관을 맺고 있으며, 엄밀한 논리와 특유의 추상성, 보편성에 의해 다른 학문들과 구별된다. 특히 수학은 과학의 여느 분야들과는 달리 자연계에서 관측되지 않는 개념들에 대해서까지 이론을 추상화시키는 특징을 보이는데, 수학자들은 그러한 개념들에 대한 추측을 제시하고 적절하게 선택된 정의와 공리로부터 엄밀한 연역을 거쳐 그 진위를 파악한다.
수학의 개념들은 기원전 600년 경에 활동하며 최초의 수학자로도 여겨지는 탈레스의 기록은 물론, 다른 고대 문명들에서도 찾아볼 수 있으며 인류의 문명과 함께 발전해 왔다. 오늘날 수학은 자연과학, 사회과학, 공학, 의학 등 거의 모든 학문에서도 핵심적인 역할을 하며 다양한 방식으로 응용된다.
수학을 의미하는 mathematics라는 단어는 '아는 모든 것', '배우는 모든 것'이라는 뜻의 고대 그리스어 'máthēma'(μάθημα) 및 그 활용형 mathēmatikós(μαθηματικός)에서 유래되었다.

---- score: 103.75523376464844 ----
수학 상수
수학에서 상수란 그 값이 변하지 않는 불변량으로, 변수의 반대말이다. 물리 상수와는 달리, 수학 상수는 물리적 측정과는 상관없이 정의된다.
수학 상수는 대개 실수체나 복소수체의 원소이다. 우리가 이야기할 수 있는 상수는 (거의 대부분 계산 가능한) 정의가능한 수이다.
특정 수학 상수, 예를 들면 골롬-딕맨 상수, 프랑세즈-로빈슨 상수, formula_1, 레비 상수와 같은 상수는 다른 수학상수 또는 함수와 약한 상관관계 또는 강한 상관관계를 갖는다.



## 8. SentenceTransformers 검색 실습
- SentenceTransformers를 이용해 관련 문서를 검색 기능을 직접 구현하는 과정입니다.

### 8.1. SentenceTransformers를 위한 전처리
- CHUNK_FN의 전체 문서를 사용하세요.

In [36]:
# SentenceTransformer 모델 생성
model = SentenceTransformer(MODEL_ID)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [37]:
# chunk embeddings 생성
chunk_embeddings = model.encode(chunk_list)
chunk_embeddings.shape

(350, 768)

### 8.2. SentenceTransformers 검색 함수
- SentenceTransformers를 활용한 검색 기능을 함수로 구현하고 실험합니다.

In [38]:
def query_sentence_transformer(model, chunk_embeddings, query, top_n=10):
    query_embedding = model.encode([query])
    # score 계산
    doc_scores = np.matmul(chunk_embeddings, query_embedding.T)
    doc_scores = doc_scores.reshape(-1)
    # score 순서로 정렬
    rank = np.argsort(-doc_scores)
    # top-n
    result = []
    for i in rank[:top_n]:
        result.append((i, doc_scores[i]))
    return result

In [39]:
while True:
    query = input('검색 > ')
    query = query.strip()
    if len(query) == 0:
        break
    result = query_sentence_transformer(model, chunk_embeddings, query)
    for i, score in result:
        print(f'---- score: {score} ----')
        print(chunk_list[i])
        print()

검색 > 수학의 정의
---- score: 161.3736114501953 ----
수학
수학(, , math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다. 널리 받아들여지는 명확한 정의는 없으나 현대 수학은 일반적으로 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 규칙의 발견과 문제의 제시 및 해결의 과정으로 이루어진다. 수학은 그 발전 과정에 있어서 철학, 과학과 깊은 연관을 맺고 있으며, 엄밀한 논리와 특유의 추상성, 보편성에 의해 다른 학문들과 구별된다. 특히 수학은 과학의 여느 분야들과는 달리 자연계에서 관측되지 않는 개념들에 대해서까지 이론을 추상화시키는 특징을 보이는데, 수학자들은 그러한 개념들에 대한 추측을 제시하고 적절하게 선택된 정의와 공리로부터 엄밀한 연역을 거쳐 그 진위를 파악한다.
수학의 개념들은 기원전 600년 경에 활동하며 최초의 수학자로도 여겨지는 탈레스의 기록은 물론, 다른 고대 문명들에서도 찾아볼 수 있으며 인류의 문명과 함께 발전해 왔다. 오늘날 수학은 자연과학, 사회과학, 공학, 의학 등 거의 모든 학문에서도 핵심적인 역할을 하며 다양한 방식으로 응용된다.
수학을 의미하는 mathematics라는 단어는 '아는 모든 것', '배우는 모든 것'이라는 뜻의 고대 그리스어 'máthēma'(μάθημα) 및 그 활용형 mathēmatikós(μαθηματικός)에서 유래되었다.

---- score: 156.95562744140625 ----
삼각함수 항등식
을 증명한다. 그리고, 미분의 극한 정의와 덧셈정리를 이용한다. 삼각함수가 테일러 급수로 정의되었다면, 각 항을 미분하여 알아낼 수 있다.
(참고 formula_92
나머지 삼각함수의 미분은 위 항등식과 미분법칙으로 얻어진다.
적분식은 적분표를 참고하라.

---- score: 156.90554809570312 ----
수학
오늘날 수학은 자연과학, 공학뿐만 아니라, 경제학 등의 사회과학에서도 중요한 도구로 사용된다. 예를 들어, 정도의 차이는 있으